# Spin Glass Generator with Random Connectivty

## Main Idea

We want to implement a smart method to enable different connectivity without boilerplate, using an adjacency matrix.
The next step should be to use a different method to compute the energy, more efficient.

In [ ]:
from typing import Optional, Union, List, Dict, Tuple

import numpy as np
from numba import jit
import matplotlib.pyplot as plt

In [ ]:
class Adjacency:
    def __init__(self):
        self.AdjDict: Dict[Tuple(int, int), float] = {}

        self.Neighbours: Optional[np.ndarray] = None
        self.SpinSide: Optional[int] = None
        self.Connectivity: Union[int, List[int]] = None
        self.MaxNeighbours: Optional[int] = None

    def create_adjacency(self, spin_side: int, connectivity: Union[int, List[int]], seed: int = 12345):
        assert connectivity <= 7, "Not implemented for connectivity greater than 7"
        self.SpinSide = spin_side
        self.Connectivity = connectivity
        self._create_adjacency(spin_side, connectivity, seed=seed)
        self._create_neighbours()

    def _create_neighbours(self):
        self.Neighbours = np.zeros((self.SpinSide**2, self.MaxNeighbours, 2))
        for spins, coupling in self.AdjDict.items():
            num_nghb = np.where(self.Neighbours[spins[0], :, 1]==0)[0]
            self.Neighbours[spins[0], num_nghb[0]] = spins[1], coupling
            num_nghb = np.where(self.Neighbours[spins[1], :, 1]==0)[0]
            self.Neighbours[spins[1], num_nghb[0]] = spins[0], coupling

    def get_adjadict(self) -> Dict[tuple, int]:
        return self.AdjDict
    
    def get_neighbours(self) -> np.ndarray:
        return self.Neighbours

    def _create_adjacency(self, spin_side: int, connectivity: Union[int, List[int]], seed: int = 12345):
        if isinstance(connectivity, int):
            connectivity = np.arange(connectivity) + 1

        # get the number of neighbours
        neighbs = np.zeros(connectivity[-1], dtype=int)
        neighbs[connectivity - 1] = 4

        # if connectivity is 4 or 7 we have 4 more neighbours
        neighbs[np.logical_and(connectivity %3 == 0, connectivity != 0)] *= 2
        self.MaxNeighbours = neighbs.max()

        # set a seed to sample couplings
        np.random.seed(seed)
        for i in range(spin_side):
            for j in range(spin_side):
                spin_num = j + i*spin_side
                # fill Adjacency dictionary
                self._create_couplings(np.asarray([i,j]), spin_num, spin_side, connectivity, seed)

    def _create_couplings(self, idxs: np.ndarray, spin_num: int, spin_side: int, connectivity: Union[int, List[int]], seed: int):
        for connect in connectivity:
            if connect == 1:
                # right spin coupling
                if idxs[1] + 1 < spin_side:
                    self.AdjDict.update({(spin_num, spin_num + 1): np.random.normal()})
                # down spin coupling
                if idxs[0] + 1 < spin_side:
                    self.AdjDict.update({(spin_num, spin_num + spin_side): np.random.normal()})
            if connect == 2:
                # up-right spin coupling
                if (idxs + [-1, 1] != [-1, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num - spin_side + 1): np.random.normal()})
                if (idxs + [1,1] != [spin_side, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num + spin_side + 1): np.random.normal()})
            if connect == 3:
                # 2-right spin coupling
                if idxs[1] + 2 < spin_side:
                    self.AdjDict.update({(spin_num, spin_num + 2): np.random.normal()})
                # 2-down spin coupling
                if idxs[0] + 2 != spin_side:
                    self.AdjDict.update({(spin_num, spin_num + 2*spin_side): np.random.normal()})
            if connect == 4:
                if (idxs + [-2, 1] != [-1, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num - 2*spin_side + 1): np.random.normal()})
                if (idxs + [-1, 2] != [-1, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num - spin_side + 2): np.random.normal()})
                if (idxs + [1, 2] != [spin_side, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num + spin_side + 2): np.random.normal()})
                if (idxs + [2, 1] != [spin_side, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num + 2*spin_side + 1): np.random.normal()})
            if connect == 5:
                if idxs[1] + 3 < spin_side:
                    self.AdjDict.update({(spin_num, spin_num + 3): np.random.normal()})
                if idxs[0] + 3 < spin_side:
                    self.AdjDict.update({(spin_num, spin_num + 3*spin_side): np.random.normal()})
            if connect == 6:
                if (idxs[0] - 2 < -1) and (idxs[1] + 2 < spin_side):
                    self.AdjDict.update({(spin_num, spin_num - 2*spin_side + 2): np.random.normal()})
                if (idxs + [2,2] < [spin_side, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num + 2*spin_side + 2): np.random.normal()})
            if connect == 7:
                if (idxs[0] - 3 > -1) and  (idxs[1] - 3 < spin_side):
                    self.AdjDict.update({(spin_num, spin_num - 3*spin_side + 1): np.random.normal()})
                if (idxs[0] - 1 > -1) and (idxs[1] + 3 < spin_side):
                    self.AdjDict.update({(spin_num, spin_num - spin_side + 3): np.random.normal()})
                if (idxs + [1, 3] < [spin_side, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num + spin_side + 3): np.random.normal()})
                if (idxs + [3, 1] < [spin_side, spin_side]).all():
                    self.AdjDict.update({(spin_num, spin_num + 3*spin_side + 1): np.random.normal()})


    def create_from_dict(self, adjacency_dict: Dict[tuple, int]):
        assert isinstance(adjacency_dict, Dict)

In [ ]:
emanuè = Adjacency()
emanuè.create_adjacency(3, 1)
emanuè.get_neighbours()

In [ ]:
emanuè = Adjacency()
emanuè.create_adjacency(20, 1)

couplings = []
for value in emanuè.get_adjadict().values():
    couplings.append(value)

In [ ]:
print(len(couplings))

plt.hist(np.asarray(couplings))

In [ ]:
@jit(nopython=True)
def compute_eng_open(Lx: int, J: np.ndarray, S0: np.ndarray) -> float:
    energy = 0.0
    for kx in range(Lx):
        for ky in range(Lx):
            k = kx + (Lx * ky)
            kR = k - ky  # coupling to the right of S0[kx,ky]
            kD = k  # coupling to the down of S0[kx,ky]

            # Tries to find a spin to right, if no spin energy contribution is 0.
            Rs = S0[kx + 1, ky] * J[kR, 0] if (kx + 1) % Lx != 0 else 0
            # Tries to find a spin to left, if no spin energy contribution is 0.
            Ds = S0[kx, ky + 1] * J[kD, 1] if (ky + 1) % Lx != 0 else 0

            energy += -S0[kx, ky] * (Rs + Ds)
    return energy / (Lx ** 2)

In [ ]:
@jit(nopython=True)
def compute_prob(eng: float, beta: float, num_spin: int) -> float:
    """Boltzmann probability distribution

    Args:
        eng (float): Energy of the sample.
        beta (float): Inverse temperature
        num_spin (int): Number of spins in the sample.

    Returns:
        float: Log-Boltzmann probability.
    """
    return - beta * num_spin * eng

In [ ]:
data = np.load("/home/beppe/neural-mcmc/sample-100000_size-484_2021-11-15_14_31_46.npz")

L = data["sample"].shape[-1]

np.random.seed(12345)
J = np.random.normal(size=(L**2 - L, 2))

boltz_prob = []
engs = []
for i, sample in enumerate(data["sample"]):
    eng = compute_eng_open(L, J, sample)
    engs.append(eng)
    boltz_prob.append(compute_prob(eng, beta=1., num_spin=L**2))

#print(np.exp(-new_prob))

In [ ]:
new_prob = np.asarray(boltz_prob)
print(boltz_prob[:10])

In [ ]:
weights = boltz_prob - data["log_prob"]
print(weights[:10])
weights -= np.log(np.exp(weights).sum())
print(weights[:10], weights.shape)


In [ ]:
engs = np.asarray(engs)
new_eng = (engs*np.exp(weights)).sum()
print(new_eng)

In [ ]:
new_idxs = np.random.choice(np.arange(data["sample"].shape[0]), size=10000, replace=False, p=np.exp(weights))
print(new_idxs.shape)

In [ ]:
print(data["log_prob"][new_idxs])

In [ ]:
new_data = {}
new_data.update({"sample": data["sample"][new_idxs], "log_prob": data["log_prob"][new_idxs]})

In [ ]:
np.savez("new_data", **new_data)